## mnist字体识别
### 第三篇：使用卷积神经网络实现mnist分类

我们希望通过mnist数据集完成更多的东西，在很多的教程或资料中，一般都直接使用官方提供的mnist数据包来直接进行读取与测试。对于初学者来说，可能在做了实验之后也不太明白自己究竟做了什么，尤其对于一个新的分类任务，当我们只有一堆数据集的时候，往往无从下手。因此，本篇尝试直接使用mnist图像来进行训练和测试。

---
首先我们观察之前创建的数据集，一共有10个文件夹，每个文件夹下对应一类图像。我们开始的第一步就是需要读取这些图像，并载入标签，也就是要手工实现keras的mnist.load_data()函数.  
**另外，在读取数据之前，我们先从图像集中剪切一部分图像出来，用作最后的预测，例如我从10个文件夹中分别抽取10张图片。**

In [1]:
import cv2
import numpy as np
import os

def get_image_label(dataset = "dataset"):
    image_list = []
    label_list = []
    
    for image_dir in os.listdir(dataset):
        for image_file in os.listdir(os.path.join(dataset, image_dir)):
            image = cv2.imread(os.path.join(dataset,image_dir,image_file), 0)
            image = np.expand_dims(image, axis=-1)
            image = image / 255.
            label = int(image_dir)
            
            image_list.append(image)
            label_list.append(label)
            
    x = np.array(image_list)
    y = np.array(label_list)
    
    return x, y

我们定义了一个get_image_label函数，该函数可以读取dataset文件夹下的文件，返回图像内容和标签。  

第二步。我们可以开始定义网络模型了。

In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend


# lenet-5网络模型；
def create_model():
    model = Sequential()
    model.add(Conv2D(20, kernel_size=5, padding="same",input_shape=(28,28,1)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(50, kernel_size=5, padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # Flatten层用于从卷积层到全连接层的过渡；
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    model.add(Dense(10))
    model.add(Activation("softmax"))
    
    return model

Using TensorFlow backend.
/usr/local/python3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


上面的代码将定义经典的lenet-5 CNN结构。  

第三步，准备读入数据，开始训练。  

*tips:除了我们事先剪切出来的一小部分用于预测的图片，我们的get_image_label函数将返回所有图片的内容和标签。而在模型评估时需要用到不参与训练的数据进行测试，我们将使用一个强大的划分训练集测试集的工具：train_test_split()*

In [6]:
from sklearn.model_selection import train_test_split

x, y = get_image_label()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1428)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

我们可以输出查看一下每个变量的纬度，如下：

In [7]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(59937, 28, 28, 1) (9986, 28, 28, 1) (59937, 10) (9986, 10)


接下来就是训练和评估模型：

In [5]:
model = create_model()
model.summary()

# 编译网络，从而交给keras后端执行；
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2450)              0         
__________

In [8]:
# 模型的训练和优化；
history = model.fit(x_train, y_train, batch_size=128, epochs=5, 
                    verbose=1, validation_split=0.2)

Train on 47949 samples, validate on 11988 samples
Epoch 1/5
47949/47949 [==============================] - 107s 2ms/step - loss: 0.1873 - acc: 0.9430 - val_loss: 0.0744 - val_acc: 0.9773
Epoch 2/5
47949/47949 [==============================] - 103s 2ms/step - loss: 0.0486 - acc: 0.9845 - val_loss: 0.0426 - val_acc: 0.9865
Epoch 3/5
47949/47949 [==============================] - 104s 2ms/step - loss: 0.0343 - acc: 0.9892 - val_loss: 0.0345 - val_acc: 0.9892
Epoch 4/5
47949/47949 [==============================] - 104s 2ms/step - loss: 0.0250 - acc: 0.9918 - val_loss: 0.0325 - val_acc: 0.9897
Epoch 5/5
47949/47949 [==============================] - 104s 2ms/step - loss: 0.0164 - acc: 0.9949 - val_loss: 0.0400 - val_acc: 0.9886


In [9]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])

9986/9986 [==============================] - 7s 700us/step
Test score: 0.04202176826189928
Test accuracy: 0.9880833166433006


In [10]:
# 保存模型；
model.save("mnist_cnn.h5")

模型保存后会在目录下生存一个model_cnn.h5的文件，保留了训练好的权重。  
我们新建一个模型，并对图像进行预测。

In [12]:
new_model = create_model()
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2450)              0         
__________

此时我们先直接预测一下图像类别，有如下几幅图像：

<figure class="third">
    <img src="0.bmp">
    <img src="1.bmp">
    <img src="2.bmp">
</figure>

In [21]:
# 预处理图像；
def pre_process(image_path):
    image = cv2.imread(image_path, 0)
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)
    image = image / 255.
    
    return image.astype("float32")

print(new_model.predict_classes(pre_process("0.bmp")))
print(new_model.predict_classes(pre_process("1.bmp")))
print(new_model.predict_classes(pre_process("2.bmp")))

[7]
[7]
[7]


我们载入训练好的模型，再次测试观察其准确率

In [23]:
# 调用load_weights载入模型；
new_model.load_weights("mnist_cnn.h5")

print(new_model.predict_classes(pre_process("0.bmp")))
print(new_model.predict_classes(pre_process("1.bmp")))
print(new_model.predict_classes(pre_process("2.bmp")))

[0]
[1]
[2]


可以看到，测试结果正确，可自己多测试几幅图像。